# Sentiment Analysis with RNN

In [18]:
# Import and data read

import numpy as np
import tensorflow as tf

with open('reviews.txt', 'r') as f:
  reviews = f.read()
with open('labels.txt', 'r') as f:
  labels = f.read()

In [15]:
reviews[:200]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  '

In [19]:
# Removing punctuation

from string import punctuation

all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

all_text[:200]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s '

In [20]:
# Word examples

words[:5]

['bromwell', 'high', 'is', 'a', 'cartoon']

In [46]:
# Word2int dictionary
vocab_to_int = {k:v for v, k in enumerate(set(words), start = 1)}

# Reviews to integers
reviews_ints = [review.split(' ') for review in reviews]

In [ ]:
!git add .
!git commit -m ""